In [ ]:
!pip install tensorflow pandas

In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense

# Specify the path to your mounted Google Drive and file
data_path = "denialpoc.csv"

# Load data using pandas
data = pd.read_csv(data_path)

# Separate features and labels
denial_reasons = data["DenialReason"]
denial_codes = data["DenialCode"]


In [ ]:
# Create a tokenizer to convert text to numerical sequences
tokenizer = Tokenizer(num_words=5000)  # Adjust num_words as needed
tokenizer.fit_on_texts(denial_reasons)

# Convert text to sequences of integers
denial_reasons_sequences = tokenizer.texts_to_sequences(denial_reasons)

# Pad sequences to have the same length
max_length = 100  # Adjust max_length as needed
denial_reasons_padded = pad_sequences(denial_reasons_sequences, maxlen=max_length)


In [ ]:
# Create a tokenizer to convert text to numerical sequences
tokenizer = Tokenizer(num_words=5000)  # Adjust num_words as needed
tokenizer.fit_on_texts(denial_reasons)

# Convert text to sequences of integers
denial_reasons_sequences = tokenizer.texts_to_sequences(denial_reasons)

# Pad sequences to have the same length
max_length = 100  # Adjust max_length as needed
denial_reasons_padded = pad_sequences(denial_reasons_sequences, maxlen=max_length)


In [ ]:
from tensorflow.keras.layers import Dropout


In [ ]:
# Convert string labels to integers (if necessary)
if pd.api.types.is_string_dtype(denial_codes):
  # Create a dictionary to map unique labels to integers
  label_mapping = {label: i for i, label in enumerate(denial_codes.unique())}

  # Convert labels to integers using the mapping
  denial_codes = denial_codes.apply(lambda x: label_mapping[x])


In [ ]:
# Define the model
model = Sequential()

# Embedding layer to map words to vectors
model.add(Embedding(5000, 128, input_length=max_length))

# Convolutional layer to capture local patterns
model.add(Conv1D(filters=32, kernel_size=3, activation="relu"))

# Max pooling layer to extract the most important feature
model.add(MaxPooling1D(pool_size=2))

# Flatten layer to prepare for dense layers
model.add(Flatten())

# Dense layer with dropout to prevent overfitting
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.2))

# Output layer with softmax activation for multi-class classification
model.add(Dense(len(denial_codes.unique()), activation="softmax"))

# Compile the model
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])


In [ ]:
# Split data into training and validation sets
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(denial_reasons_padded, denial_codes, test_size=0.2)

# Train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))


Epoch 1/10
40/40 [==============================] - 9s 82ms/step - loss: 3.5675 - accuracy: 0.2084 - val_loss: 2.9135 - val_accuracy: 0.3228
Epoch 2/10
40/40 [==============================] - 1s 30ms/step - loss: 2.3675 - accuracy: 0.4184 - val_loss: 1.8390 - val_accuracy: 0.5475
Epoch 3/10
40/40 [==============================] - 1s 15ms/step - loss: 1.5537 - accuracy: 0.6553 - val_loss: 1.0552 - val_accuracy: 0.7943
Epoch 4/10
40/40 [==============================] - 1s 20ms/step - loss: 0.9319 - accuracy: 0.8019 - val_loss: 0.6262 - val_accuracy: 0.8734
Epoch 5/10
40/40 [==============================] - 1s 13ms/step - loss: 0.6253 - accuracy: 0.8708 - val_loss: 0.4182 - val_accuracy: 0.9367
Epoch 6/10
40/40 [==============================] - 0s 12ms/step - loss: 0.4259 - accuracy: 0.9105 - val_loss: 0.2982 - val_accuracy: 0.9462
Epoch 7/10
40/40 [==============================] - 1s 13ms/step - loss: 0.3062 - accuracy: 0.9374 - val_loss: 0.2365 - val_accuracy: 0.9589
Epoch 8/10
40

In [ ]:
# Evaluate the model on the validation set
loss, accuracy = model.evaluate(X_val, y_val)
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)


10/10 [==============================] - 0s 3ms/step - loss: 0.1362 - accuracy: 0.9810
Validation Loss: 0.13618575036525726
Validation Accuracy: 0.9810126423835754


In [ ]:
import numpy as np

In [28]:
new_reason = "Coverage Expired"
new_reason_sequence = tokenizer.texts_to_sequences([new_reason])
new_reason_padded = pad_sequences(new_reason_sequence, maxlen=max_length)
prediction = model.predict(new_reason_padded)[0]
predicted_class_index = np.argmax(prediction)
predicted_code = list(label_mapping.keys())[predicted_class_index]
print("Predicted Denial Code:", predicted_code)

1/1 [==============================] - 0s 20ms/step
Predicted Denial Code: 27


In [29]:
save_path = '/content/drive/MyDrive/textcnn'
model.save(save_path + 'TextCNN')